In [1]:
import requests
import json
import os
from openai import OpenAI
from google.colab import userdata
import pprint

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
news_key = userdata.get('NEWS_API_KEY')

client = OpenAI()